# Homework 3

## FINM 37400 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (C): Spread and Swap Spread in November 2008

In [2]:
from datetime import datetime
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:,.2f}'.format

## Simplification of the setup

The date is Nov 4, 2008.

**Treasury bond**
* Suppose the Treasury bond matures exactly 30 years later, on Nov 4, 2038 rather than May 15, 2008. 
* The YTM of this freshly issued treasury is 4.193\% with a semiannual coupon of 4.5\%, same as is given in the case. (So we're just changing the maturity date to simplify things, but keeping the market data.)

**Swap**
* The fixed leg of the swap pays semiannually, with swap rate of 4.2560\%, as given in the case.
* The floating leg of the swap also pays semiannually--not quarterly--such that the payment dates are identical on both legs. Thus, it also resets the floating rate semiannually, not quarterly.
* The floating rate of the swap equals the repo rate used in the trade. Thus, these two rates cancel in the financing of the trade. (No need to consider the TED spread.) 

## 1.1

List the projected cashflows on May 4, 2009, exactly six months into the trade, on the first coupon and swap date.

**Answers:**

The floating leg of the swap and the Repo cancel in the cashflows. 

In [35]:
idx = ["T bond", "Repo", "Swap(floating leg)", "Swap(fixed leg)", "Net Payment"]
cashflow = [0.045/2*1000000000, np.nan, np.nan, -0.04256/2*1000000000, 0.045/2*1000000000-0.04256/2*1000000000]
CashFlow = pd.DataFrame(cashflow, columns=["Cash Flow"], index=idx)
CashFlow

,Cash Flow
T bond,"22,500,000.00"
Repo,NaN
Swap(floating leg),NaN
Swap(fixed leg),"-21,280,000.00"
Net Payment,"1,220,000.00"


## 1.2

What is the duration of...
* the T-bond
* the swap

Remember that...
* the swap can be decomposed into a fixed-rate bond and a floating-rate note
* a floating-rate note has duration equal to the time until the next reset. Thus, at initialization, it has duration equal to 0.5 years.

Is the duration for the "paying-fixed" swap positive or negative? Is it bigger or smaller in magnitude than the T-bond?

For this problem, calculate the Macauley duration and the dollar (Macauley) duration.

**Answers:**

- DV01 for $10 million swap is $17,191
- T-Bond Val01: 0.1746

In [84]:
# Use YTM to calculte duration
def bp(cashflow, ytm, n, t):
    bondprice = 0
    for i in range(1, len(cashflow)+1):
        bondprice += cashflow[i-1]/((1+ytm/n)**i)
    return bondprice

def cal_dura(coupon, ytm, n, t, bondprice=False):
    num = int(n*t)
    cashflow = np.array([coupon/n*100]*num)
    cashflow[-1] += 100
    if bondprice == False:
        bondprice = bp(cashflow, ytm, n, t)
    dura = 0
    for i in range(1, len(cashflow)+1):
        dura += i/2*cashflow[i-1]/((1+ytm/n)**i)
    return dura/bondprice

coupon = 100 * 0.045 / 2
Cashflow = np.array([coupon]*60)
Cashflow[-1] += 100
dura_t_bond = cal_dura(0.045, 0.04193, 2, 30)
bond_price = bp(Cashflow, 0.04193, 2, 30)
dura_swap_fix = cal_dura(0.04256, 0.04256, 2, 30, 100)

In [79]:
idx = ["T bond", "Swap(floating leg)", "Swap(fixed leg)", "Swap", "Net"]
dura_df = pd.DataFrame(index=idx)
TBond_dura = 17.46
Swap_dura = 17.19
TBond_price = 105.17
# bond_price = 105
dura_df["Case Reported Duration"] = [TBond_dura, 0.5, Swap_dura+0.5, Swap_dura, TBond_dura-Swap_dura]
dura_df["Case Reported Dollar Duration"] = [TBond_dura*TBond_price, 50, (Swap_dura+0.5)*100, Swap_dura*100, TBond_dura*TBond_price-Swap_dura*100]
dura_df["Calculated Duration"] = [dura_t_bond, 0.5, dura_swap_fix, dura_swap_fix-0.5, dura_t_bond-(dura_swap_fix-0.5)]
dura_df["Calculated Dollar Duration"] = [dura_t_bond*bond_price, 50, dura_swap_fix*100, (dura_swap_fix-0.5)*100, dura_t_bond*TBond_price-(dura_swap_fix-0.5)*100]
dura_df

,Case Reported Duration,Case Reported Dollar Duration,Calculated Duration,Calculated Dollar Duration
T bond,17.46,"1,836.27",17.08,"1,797.43"
Swap(floating leg),0.50,50.00,0.50,50.00
Swap(fixed leg),17.69,"1,769.00",17.21,"1,721.27"
Swap,17.19,"1,719.00",16.71,"1,671.27"
Net,0.27,117.27,0.37,125.41


## 1.3

What hedge ratio should be used to balance the notional size of the Treasury bond with the notional size of the swap, such that it is a duration-neutral position?

Specifically, if the trader enters the swap paying fixed on \$500 million notional, how large of a position should they take in the Treasury bond?

In [126]:
dura_swap = -(dura_swap_fix-0.5)
# bond_price = 105
ratio = -dura_swap*100/(dura_t_bond*bond_price)
ratio

0.9298159049268911

In [83]:
dura_swap = -(dura_swap_fix-0.5)
-500*1000000*dura_swap*100/(dura_t_bond*bond_price)

464907952.46344566

## 1.4

Suppose it is May 4, 2009, exactly six months after putting the trade on.

The spread is at -28 bps due to...
* The YTM on a new 30-year bond has risen to 4.36\%
* The swap rate on a new 30-year swap has dropped to 4.08\%

Explain conceptually how this movement impacts the components of the trade.

## 1.5

Calculate the value of the position on May 4, 2009, immediately after the first coupon and swap payments and swap reset. 

* Calculate the revised price of the Treasury bond by assuming you can apply the (May 4) 30-year YTM as a discount rate to the 29.5 year bond. (We are just using this for a rough approximation. You know that good pricing would require a discount curve, but let's not get bogged down with that here.)


* Calculate the value of the swap by decomposing it into a fixed-rate bond and a floating-rate bond.
    * The 29.5 year fixed-rate leg is priced using the (May 4) 30-year swap rate as a discount rate.
    * The floating-rate leg is priced at par given that floating-rate notes are par immediately after resets.
    
**Note**

You are being asked to calculate these valuations using the exact formula between price, cashflows, and YTM discount rate. We are not simply approximating with duration, as we already know the position was set up with zero dollar duration.

From the Discussion 1 notebook, we have this formula expressing a bond's price as a function of the coupon, $c$, and the YTM, $y_j$.

$$\begin{align*}
P_j(t,T,c) = \sum_{i=1}^{n-1}\frac{100\left(\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T_i-t)}} + \frac{100\left(1+\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T-t)}}
\end{align*}
$$

In [109]:

def pv(c_rate, ytm, n, t):
    bondprice = 0
    num = int(n*t)
    cashflow = np.array([c_rate/n*100]*num)
    cashflow[-1] += 100
    for i in range(1, len(cashflow)+1):
        bondprice += cashflow[i-1]/((1+ytm/n)**i)
    return bondprice

bond_coupon = 0.045
bond_ytm = 0.0436
T_bond_price = pv(bond_coupon, bond_ytm, 2, 29.5)

swap_coupon = 0.04256
swap_rate = 0.0408
fixed_rate_leg = pv(swap_coupon, swap_rate, 2, 29.5)
floating_rate_leg = 100
swap_price = floating_rate_leg - fixed_rate_leg

In [110]:
idx = ["T-Bond", "Fixed-leg Price", "Floating-leg Price", "Swap Price"]
value = [T_bond_price, fixed_rate_leg, floating_rate_leg, swap_price]
pd.DataFrame(data=value, columns=["May 4 2009"], index=idx)

,May 4 2009
T-Bond,102.31
Fixed-leg Price,103.00
Floating-leg Price,100.00
Swap Price,-3.00


In [125]:
swap_coupon = 0.04256
swap_rate = 0.04256
fixed_rate_leg = pv(swap_coupon, swap_rate, 2, 29.5)
fixed_rate_leg

100.00000000000011

## 1.6

Accounting for the change in value of the positions, as well as the 6-month cashflows paid on May 4, 
* what is the net profit and loss (pnl) of the position?
* what is the return on the equity capital, considering that there was a 2\% haircut (equity contribution) on the size of the initial treasury bond position.

In [113]:
ratio

-0.9298159049268911

In [137]:
cf = (bond_coupon * ratio - swap_coupon) * 100 / 2 
price_change = (T_bond_price - bond_price) * ratio + (swap_price - 0)
pnl = cf + price_change
pnl = pnl * 500 * 1000000 / 100
pnl

-28687393.94191092

In [138]:
bond_pos = -500*1000000*dura_swap*100/(dura_t_bond*bond_price)
pnl / (bond_pos * 0.02)

-3.0852767510108916

In [136]:
bond_pos

464907952.46344566